In [1]:
!pip install tensorflow keras numpy matplotlib scikit-learn opencv-python


  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ----


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [3]:
X, Y = [], []
image_size = 128
data_dir = "data"  # Change this if your folder path is different

for fname in os.listdir(data_dir):
    if fname.lower().endswith((".jpg", ".jpeg", ".png")):
        label = fname.split("_")[0]
        path = os.path.join(data_dir, fname)
        img = cv2.imread(path)
        img = cv2.resize(img, (image_size, image_size))
        img = img / 255.0
        X.append(img)
        Y.append(label)

X = np.array(X)
Y = np.array(Y)
print("Images loaded:", X.shape)
print("Unique Labels:", np.unique(Y))


Images loaded: (790, 128, 128, 3)
Unique Labels: ['anadenanthera' 'arecaceae' 'arrabidaea' 'cecropia' 'chromolaena'
 'combretum' 'croton' 'dipteryx' 'eucalipto' 'faramea' 'hyptis' 'mabea'
 'matayba' 'mimosa' 'myrcia' 'protium' 'qualea' 'schinus'
 'senegalia (1).jpg' 'senegalia (10).jpg' 'senegalia (11).jpg'
 'senegalia (12).jpg' 'senegalia (13).jpg' 'senegalia (14).jpg'
 'senegalia (15).jpg' 'senegalia (16).jpg' 'senegalia (17).jpg'
 'senegalia (18).jpg' 'senegalia (19).jpg' 'senegalia (2).jpg'
 'senegalia (20).jpg' 'senegalia (21).jpg' 'senegalia (22).jpg'
 'senegalia (23).jpg' 'senegalia (24).jpg' 'senegalia (25).jpg'
 'senegalia (26).jpg' 'senegalia (27).jpg' 'senegalia (28).jpg'
 'senegalia (29).jpg' 'senegalia (3).jpg' 'senegalia (30).jpg'
 'senegalia (31).jpg' 'senegalia (32).jpg' 'senegalia (33).jpg'
 'senegalia (34).jpg' 'senegalia (35).jpg' 'senegalia (4).jpg'
 'senegalia (5).jpg' 'senegalia (6).jpg' 'senegalia (7).jpg'
 'senegalia (8).jpg' 'senegalia (9).jpg' 'serjania' 'syag

In [4]:
encoder = LabelEncoder()
Y_enc = encoder.fit_transform(Y)
Y_cat = to_categorical(Y_enc)

X_train, X_test, y_train, y_test = train_test_split(
    X, Y_cat, test_size=0.2, random_state=42
)
num_classes = Y_cat.shape[1]

print("Train/Test split:", X_train.shape[0], "train–", X_test.shape[0], "test")


Train/Test split: 632 train– 158 test


In [5]:
model = Sequential([
    Conv2D(16, (3,3), activation='relu', padding='same', input_shape=(image_size, image_size, 3)),
    MaxPooling2D(2,2),
    Conv2D(32, (2,2), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Conv2D(64, (2,2), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Conv2D(128, (2,2), activation='relu', padding='same'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.2),
    Dense(500, activation='relu'),
    Dense(150, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\mbala\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 128, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 32)          │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 64)          │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 16, 128)         │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 500)                 │       4,096,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 150)                 │          75,150 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 91)                  │          13,741 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,229,071 (16.13 MB)

 Trainable params: 4,229,071 (16.13 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)


Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 197ms/step - accuracy: 0.0384 - loss: 4.2281 - val_accuracy: 0.0394 - val_loss: 3.6010
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.0342 - loss: 3.6168 - val_accuracy: 0.0551 - val_loss: 3.4144
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 131ms/step - accuracy: 0.0876 - loss: 3.4048 - val_accuracy: 0.1102 - val_loss: 3.2496
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.1437 - loss: 3.1445 - val_accuracy: 0.0787 - val_loss: 3.0154
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 131ms/step - accuracy: 0.1825 - loss: 2.8603 - val_accuracy: 0.1732 - val_loss: 2.8523
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 133ms/step - accuracy: 0.2844 - loss: 2.7077 - val_accuracy: 0.3307 - val_loss: 2.6958
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step - accuracy: 0.3691 - loss: 2.2844 - val_accuracy: 0.3307 - val_loss: 2.4718
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - accuracy: 0.4076 - loss: 2.0174 - val_accuracy: 0

In [7]:
model.save("model.h5")
print("✅ model.h5 saved in your POLLEN_GRAIN folder.")


✅ model.h5 saved in your POLLEN_GRAIN folder.
